In [2]:
import matplotlib
matplotlib.use("Agg")

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras import utils as np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cancernet.cancernet2 import CancerNet
from cancernet import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.models import load_model
import time

#tf.debugging.set_log_device_placement(True)

NUM_EPOCHS=40; INIT_LR=1e-3; BS=128

#lenTrain: 255815
#lenVal: 42660
#lenTest: 99906

print(config.TRAIN_PATH)

trainPaths=list(paths.list_images(config.TRAIN_PATH))
lenTrain=len(trainPaths)
valPaths=list(paths.list_images(config.VAL_PATH))
lenVal=len(valPaths)
testPaths=list(paths.list_images(config.TEST_PATH))
lenTest=len(testPaths)

trainLabels=[int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels=np_utils.to_categorical(trainLabels)
classTotals=trainLabels.sum(axis=0)
classWeight=classTotals.max()/classTotals

trainAug = ImageDataGenerator(
	rescale=1/255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

valAug=ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

#LOAD MODEL
#model = load_model('model.h5')

model=CancerNet.build(width=48,height=48,depth=3,classes=2)
#model = load_model('model.h5')

opt=Adagrad(lr=INIT_LR,decay=INIT_LR/NUM_EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])

classWeight = {i : classWeight[i] for i in range(2)}


start=time.time()
M=model.fit(
	trainGen,
	steps_per_epoch=lenTrain//BS,
	validation_data=valGen,
	validation_steps=lenVal//BS,
	class_weight=classWeight,
	epochs=NUM_EPOCHS)
print("Passed: ", time.time()-start)

#SAVE MODEL
model.save('model-AlexNet.h5')

print("Now evaluating the model")
testGen.reset()
pred_indices=model.predict(testGen,steps=(lenTest//BS)+1)

pred_indices=np.argmax(pred_indices,axis=1)

print(classification_report(testGen.classes, pred_indices, target_names=testGen.class_indices.keys()))

cm=confusion_matrix(testGen.classes,pred_indices)
total=sum(sum(cm))
accuracy=(cm[0,0]+cm[1,1])/total
specificity=cm[1,1]/(cm[1,0]+cm[1,1])
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
print(cm)
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')

content/drive/My Drive/DeepLearn/datasets/idc\training
Found 255815 images belonging to 2 classes.
Found 42660 images belonging to 2 classes.
Found 99906 images belonging to 2 classes.
Epoch 1/40
1998/1998 [==============================] - 453s 227ms/step - loss: 0.7428 - accuracy: 0.7843 - val_loss: 0.4302 - val_accuracy: 0.8131
Epoch 2/40
1998/1998 [==============================] - 441s 221ms/step - loss: 0.6374 - accuracy: 0.8084 - val_loss: 0.4457 - val_accuracy: 0.8107
Epoch 3/40
1998/1998 [==============================] - 480s 240ms/step - loss: 0.6098 - accuracy: 0.8164 - val_loss: 0.4402 - val_accuracy: 0.8091
Epoch 4/40
1998/1998 [==============================] - 448s 224ms/step - loss: 0.5902 - accuracy: 0.8234 - val_loss: 0.4312 - val_accuracy: 0.8155
Epoch 5/40
1998/1998 [==============================] - 524s 262ms/step - loss: 0.5823 - accuracy: 0.8253 - val_loss: 0.4284 - val_accuracy: 0.8243
Epoch 6/40
1998/1998 [==============================] - 460s 230ms/step - l

In [ ]:
import matplotlib.pyplot as plt
N = 40
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N), M.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), M.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), M.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0,N), M.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on the IDC Dataset")
plt.xlabel("Epoch No.")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('plot.png')

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras import utils as np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cancernet.cancernet2 import CancerNet
from cancernet import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.models import load_model
import time


BS=128

trainPaths=list(paths.list_images(config.TRAIN_PATH))
lenTrain=len(trainPaths)
valPaths=list(paths.list_images(config.VAL_PATH))
lenVal=len(valPaths)
testPaths=list(paths.list_images(config.TEST_PATH))
lenTest=len(testPaths)

trainLabels=[int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels=np_utils.to_categorical(trainLabels)
classTotals=trainLabels.sum(axis=0)
classWeight=classTotals.max()/classTotals

trainAug = ImageDataGenerator(
	rescale=1/255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

valAug=ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)


model=CancerNet.build(width=48,height=48,depth=3,classes=2)
model = load_model('model-AlexNet.h5')

classWeight = {i : classWeight[i] for i in range(2)}

pred_indices=model.predict(testGen,steps=(lenTest//BS)+1)

pred_indices=np.argmax(pred_indices,axis=1)

print(classification_report(testGen.classes, pred_indices, target_names=testGen.class_indices.keys()))

cm=confusion_matrix(testGen.classes,pred_indices)
total=sum(sum(cm))
accuracy=(cm[0,0]+cm[1,1])/total
specificity=cm[1,1]/(cm[1,0]+cm[1,1])
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
print(cm)
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')

Found 255815 images belonging to 2 classes.
Found 42660 images belonging to 2 classes.
Found 99906 images belonging to 2 classes.
              precision    recall  f1-score   support

           0       0.95      0.81      0.87     71451
           1       0.65      0.88      0.75     28455

    accuracy                           0.83     99906
   macro avg       0.80      0.84      0.81     99906
weighted avg       0.86      0.83      0.84     99906

[[57690 13761]
 [ 3351 25104]]
Accuracy: 0.8287189958561048
Specificity: 0.8822351080653663
Sensitivity: 0.8074064743670487


In [10]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (48, 48, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image
image1 = load('0.png')
image2 = load('1.png')
print(np.argmax(model.predict(image1),axis=1))
print(np.argmax(model.predict(image2),axis=1))

[0]
[1]
